In [154]:
import scipy
import pandas as pd
import numpy as np
from numpy.linalg import norm
import cmath
import warnings
warnings.filterwarnings("ignore")

In [142]:
# Givens Rotation
def givens(a:int, b:int, hyperbolic:bool=False):
    if hyperbolic:
        r = np.sqrt(a**2 - b**2)
    else:
        r = np.sqrt(a**2 + b**2)
    c, s = a/r, b/r 
    return c, s


def permu_matrix(size:int, inverse:bool=False):
    if inverse:
        C = np.eye(size-2)
        C = np.vstack((np.zeros(size-2), C))
        C = np.hstack((C, np.asmatrix(np.zeros(size-1)).T))
        C[0, -1] = 1
        return C
    B = np.eye(size)
    B[0, 0], B[-1, -1] = 0, 0
    B[0, -1], B[-1, 0] = 1, 1
    return B


# QR factorization based on givens
def qr_givens(A:np.array, reduced:bool=False):
    m, n = A.shape
    Q = np.eye(m)
    R = np.copy(A)

    for j in range(n):
        for i in range(m-1, j, -1):
            c, s = givens(R[i-1,j], R[i,j])
            R[i-1, :], R[i, :] = c*R[i-1, :] + s*R[i, :], (-s)*R[i-1, :] + c*R[i, :]
            Q[:, i-1], Q[:, i] = c*Q[:, i-1] + s*Q[:, i], (-s)*Q[:, i-1] + c*Q[:, i]
    if reduced:
        return Q[:, :n], R[:n, :]
    else:
        return Q, R


In [143]:
# test 
A= np.random.rand(5,3)*10
Q_r, R_r = qr_givens(A, reduced=True)
print("Given A:\n", A)
print("Reduced Q matrix:\n", Q_r)
print("Reduced R matrix:\n", R_r)
print("Q @ R:\n", (Q_r @ R_r))

Given A:
 [[4.48130979 3.10323631 5.03276062]
 [0.29125743 8.40402502 2.13098582]
 [4.40528802 8.66665188 9.84080928]
 [1.46868062 5.50716828 7.56202559]
 [6.75220507 2.45376289 5.84866533]]
Reduced Q matrix:
 [[ 0.47955793 -0.0877351  -0.09179494]
 [ 0.0311683   0.74336517 -0.6251004 ]
 [ 0.47142262  0.42663253  0.26997457]
 [ 0.15716776  0.38130142  0.68664492]
 [ 0.722573   -0.33511892 -0.23760403]]
Reduced R matrix:
 [[9.34466837e+00 8.47434859e+00 1.25336938e+01]
 [4.44008642e-17 1.09500611e+01 6.26437297e+00]
 [4.64554632e-17 8.25966346e-16 4.66546619e+00]]
Q @ R:
 [[4.48130979 3.10323631 5.03276062]
 [0.29125743 8.40402502 2.13098582]
 [4.40528802 8.66665188 9.84080928]
 [1.46868062 5.50716828 7.56202559]
 [6.75220507 2.45376289 5.84866533]]


In [144]:
def full_qr_del(A): # Full QR Downdating
    m, n = A.shape
    Q, R = qr_givens(A)
    
    B = permu_matrix(m)
    Q = B @ Q

    q1 = np.asmatrix(Q[0, :]).T
    J, _ = qr_givens(q1)
    Q_ = Q @ J
    q1_R = np.hstack((q1, R))
    R1 = (J.T @ q1_R)[1:, 1:]
    Q1 = Q_[1:, 1:]

    C = permu_matrix(m, inverse=True)
    Q1 = C @ Q1


    return Q1, R1

In [171]:
def Full_QR_Down(A, b):
    m, n = A.shape
    Q, R = full_qr_del(A)
    b = b[:-1]

    y = (Q.T @ b).T
    x = np.asarray(np.linalg.solve(R[:n, :], y[:n]))
    x = np.squeeze(np.asarray(x))
    return x

In [146]:
# test
A= np.random.rand(5,3)*10
print("Given a matrix A:", "\n", A, "\n")
Q1, R1 = full_qr_del(A)
print("Q_ matrix of A deleteing the first row(A_):\n", Q1, "\n", "R_ matrix of A deleteing the first row(A_):\n", R1, "\n")
q, r = np.linalg.qr(A[:-1, :], mode='complete')
print("q:\n", Q1, "\n", "r:\n", R1, "\n")
print("Test if A_ = Q_ @ R_: \n", (Q1@R1), "\n")

Given a matrix A: 
 [[9.64729562 8.21624414 8.67647521]
 [7.40641247 2.47585275 9.25046287]
 [9.73028239 3.67422319 7.95250109]
 [3.68905104 7.25888703 4.92206976]
 [0.8584382  7.4605301  9.83460058]] 

Q_ matrix of A deleteing the first row(A_):
 [[-0.60270559 -0.34685438  0.30878403  0.64891482]
 [-0.46270855  0.34151904 -0.80660048  0.13660605]
 [-0.60789011  0.38181302  0.41581207 -0.55838048]
 [-0.23046994 -0.78566889 -0.28486561 -0.4984572 ]] 
 R_ matrix of A deleteing the first row(A_):
 [[-1.60066471e+01 -1.00040537e+01 -1.54782642e+01]
 [-3.78188306e-16 -6.30450487e+00 -6.81012881e-01]
 [-1.81805403e-16  4.66756035e-16 -2.87765332e+00]
 [-1.10963609e-16  1.11939546e-16  4.33217362e-16]] 

q:
 [[-0.60270559 -0.34685438  0.30878403  0.64891482]
 [-0.46270855  0.34151904 -0.80660048  0.13660605]
 [-0.60789011  0.38181302  0.41581207 -0.55838048]
 [-0.23046994 -0.78566889 -0.28486561 -0.4984572 ]] 
 r:
 [[-1.60066471e+01 -1.00040537e+01 -1.54782642e+01]
 [-3.78188306e-16 -6.304504

In [147]:
def reduced_qr_del(A): # Reduced QR Downdating
    m, n = A.shape
    Q1, R1 = qr_givens(A, reduced=True)

    B = permu_matrix(m)
    Q1 = B @ Q1
    
    e1 = np.asmatrix(np.eye(m)[:, 0]).T
    q1 = np.asmatrix(Q1[0, :]).T
    v = e1 - Q1 @ q1
    if norm(v) >= np.sqrt(1/2):
        v = v/norm(v)
    else:
        s = Q1.T @ v
        v_ = v - Q1 @ s
        if norm(v_) >= norm(v)/np.sqrt(2):
            v = v_/norm(v_)
        else:
            h = (Q1[1:, 1:]).nullspace()[0]
            h = h/norm(h)
            v = np.hstack((np.zeros(1), h))
    Q1_e1 = np.hstack((Q1, v))

    # givens rotation to eliminate first row
    #J = np.eye(n+1)
    R_0 = np.vstack((R1, np.zeros(n)))
    for i in range(n-1, -1, -1):
        c, s = givens(Q1_e1[0, n], Q1_e1[0, i])
        Q1_e1[:, n], Q1_e1[:, i] = c*Q1_e1[:, n] + s*Q1_e1[:, i], (-s)*Q1_e1[:, n] + c*Q1_e1[:, i]
        R_0[n, :], R_0[i, :] = c*R_0[n, :] + s*R_0[i, :], (-s)*R_0[n, :] + c*R_0[i, :]
    
    Q = Q1_e1[1: ,:-1]
    R = R_0[:-1, :]

    C = permu_matrix(m, inverse=True)
    Q = C @ Q

    return Q, R
        

In [148]:
def Reduced_QR_Down(A, b): # Reduced QR Downdating
    A_b = np.hstack((A, np.asmatrix(b).T))
    Q, R = reduced_qr_del(A_b)
    R_, u = R[:-1, :-1], R[:-1, -1]

    x = np.linalg.solve(R_, u)
    return x
        

In [149]:
# test
A= np.random.rand(5,3)*10
print("Given a matrix A:", "\n", A, "\n")
Q1, R1 = reduced_qr_del(A)
print("Q_ matrix of A deleteing the first row(A_):\n", Q1, "\n", "R_ matrix of A deleteing the first row(A_):\n", R1, "\n")
q, r = np.linalg.qr(A[:-1, :], mode='reduced')
print("q:\n", Q1, "\n", "r:\n", R1, "\n")
print("Test if A_ = Q_ @ R_: \n", Q1@R1, "\n")

Given a matrix A: 
 [[7.07534468 1.77555431 6.11702702]
 [9.3264824  4.54896777 9.75816405]
 [6.57093765 6.70460292 7.49654724]
 [6.29512725 5.12067695 2.13701403]
 [7.48550661 2.59269952 6.82579297]] 

Q_ matrix of A deleteing the first row(A_):
 [[ 0.47718275 -0.60067458 -0.0968729 ]
 [ 0.62900632 -0.25018543  0.32418474]
 [ 0.44316401  0.681975    0.44071571]
 [ 0.42456252  0.33392725 -0.83143825]] 
 R_ matrix of A deleteing the first row(A_):
 [[14.82732697  8.85387957 13.28638264]
 [ 0.          4.07768931 -0.28962811]
 [ 0.          0.          4.09792466]] 

q:
 [[ 0.47718275 -0.60067458 -0.0968729 ]
 [ 0.62900632 -0.25018543  0.32418474]
 [ 0.44316401  0.681975    0.44071571]
 [ 0.42456252  0.33392725 -0.83143825]] 
 r:
 [[14.82732697  8.85387957 13.28638264]
 [ 0.          4.07768931 -0.28962811]
 [ 0.          0.          4.09792466]] 

Test if A_ = Q_ @ R_: 
 [[7.07534468 1.77555431 6.11702702]
 [9.3264824  4.54896777 9.75816405]
 [6.57093765 6.70460292 7.49654724]
 [6.29512

In [150]:
def r_qr_del(A, method2=False): # Cholesky Downdatinf
    m, n = A.shape
    _, R = qr_givens(A)

    if method2:
        import cmath
        a = np.asmatrix(1j*A[-1, :])
        R_ = np.vstack((R, a))
        for i in range(n):
            c, s = givens(R_[i,i], R_[m-1,i])
            R_[i, :], R_[m-1, :] = c*R_[i, :] + s*R_[m-1, :], (-s)*R_[i, :] + c*R_[m-1, :]
        return R_


    a = np.asmatrix(A[-1, :])
    R_ = np.vstack((R, a))
    for i in range(n):
        ch, sh = givens(R_[i,i], R_[m,i], hyperbolic=True)
        R_[i, :], R_[m, :] = ch*R_[i, :] + (-sh)*R_[m, :], (-sh)*R_[i, :] + ch*R_[m, :]
    return R_[:-2, :]
        

    

In [167]:
def R_Down(A, b):
    m, n = A.shape
    Q, _ = qr_givens(A[:-1, :])
    R = r_qr_del(A)
    b = b[:-1]

    y = (Q.T @ b).T
    x = np.linalg.solve(R[:n, :], y[:n])
    return x

In [208]:
def matrix_gen(m, n):
    cond_P = 10000**2     # Condition number
    log_cond_P = np.log(cond_P)
    exp_vec = np.arange(-log_cond_P/n, log_cond_P * (n + 1)/(n * (n - 1)), log_cond_P/((n/2)*(n-1)))
    s = np.exp(exp_vec)
    S = np.diag(s)
    U, _ = np.linalg.qr((np.random.rand(m, n) - 5.) * 200)
    V, _ = np.linalg.qr((np.random.rand(n, n) - 5.) * 200)
    P = U.dot(S).dot(V.T)
    return P

In [199]:
A = np.random.rand(5,3)*10
Q, R = qr_givens(A)
_, R1 = qr_givens(A[:-1, :])
R_ = r_qr_del(A)


print("Given a matrix A:", "\n", A, "\n")
print("Delete Last row:", "\n", A[:-1, :], "\n")

print("Result of A_.T @ A_:", "\n", A[:-1, :].T@A[:-1, :], "\n")

print("Print R1 directly):", "\n", R1, "\n")
print("Result of R1.T @ R1:", "\n", R1.T@R1, "\n")
print("Print R__ from Method1):", "\n", R_, "\n")
print("Result of R_.T @  R_ (R_ from Method1):", "\n", R_.T@R_, "\n")





Given a matrix A: 
 [[8.17023783 0.31629431 7.17299614]
 [2.20471297 9.83527128 8.57032268]
 [6.72544435 9.38165921 1.99571996]
 [8.62936846 6.37779562 0.9570518 ]
 [2.20818642 0.2480465  2.23170067]] 

Delete Last row: 
 [[8.17023783 0.31629431 7.17299614]
 [2.20471297 9.83527128 8.57032268]
 [6.72544435 9.38165921 1.99571996]
 [8.62936846 6.37779562 0.9570518 ]] 

Result of A_.T @ A_: 
 [[191.31114725 142.40032525  99.18104223]
 [142.40032525 225.52440974 111.38727169]
 [ 99.18104223 111.38727169 129.80115074]] 

Print R1 directly): 
 [[13.83152729 10.29534355  7.17065008]
 [ 0.         10.93299186  3.43574441]
 [-0.          0.          8.15957037]
 [ 0.          0.          0.        ]] 

Result of R1.T @ R1: 
 [[191.31114725 142.40032525  99.18104223]
 [142.40032525 225.52440974 111.38727169]
 [ 99.18104223 111.38727169 129.80115074]] 

Print R__ from Method1): 
 [[ 1.38315273e+01  1.02953435e+01  7.17065008e+00]
 [ 5.59789297e-17  1.09329919e+01  3.43574441e+00]
 [-8.19872123e-17

In [209]:
A = matrix_gen(20, 6)#np.random.rand(20, 6)
b = np.random.rand(20)

A[-1, :] = 10000000*A[-1, :]
b[-1] = 10000000*b[-1]

print(A)
print(b)

print("")

print(Full_QR_Down(A, b))
print(Reduced_QR_Down(A, b))
print(R_Down(A, b))
print(np.linalg.lstsq(A[:-1, :], b[:-1])[0])

[[ 4.62160906e+00  1.32611564e+00 -5.52770854e-01 -5.49026432e+00
   7.73397765e-01 -4.35821827e-01]
 [ 8.10248441e-01 -9.30018178e-01 -5.62849679e-02 -7.04790937e-01
   1.77834492e-01  8.18381788e-01]
 [ 6.06065386e-01 -5.38261968e-01 -2.16302220e-01 -1.01069163e+00
   7.52432259e-01  5.15110542e-01]
 [-5.62680374e+00  2.60593431e+00  1.09101590e+00  6.72485684e+00
  -2.46351184e+00 -2.78676508e+00]
 [-6.03598977e+00 -2.01727435e-01  8.27570841e-01  6.76993639e+00
  -1.10987421e+00 -5.44395144e-01]
 [-1.57548487e+00  1.91814785e+00  5.21412874e-01  2.32426369e+00
  -1.58823409e+00 -1.80646766e+00]
 [-2.33493259e+00 -8.80559112e-01  3.77561679e-01  3.04048649e+00
  -5.59934514e-01  2.62919076e-01]
 [-1.70160377e+00  1.93900767e+00  3.42355031e-01  1.55819634e+00
  -4.09070967e-01 -1.88623055e+00]
 [-1.45219379e+00  1.59964580e-01 -4.41628973e-02  1.16470709e+00
   2.67435358e-01 -1.33654530e-01]
 [ 2.90176240e+00 -6.72180077e-01 -4.12154398e-01 -3.01464542e+00
   4.35546572e-01  9.7253